# Read dataset and plot graph

## Process input jsons and create metafeature file

In [ ]:
# import os
# import pandas as pd
# import json
# from pathlib import Path

# def get_dirs(path):
#     drs = os.walk(path)
#     dirs = []
#     for dr in drs:
#         dirs.append(dr)
#     return dirs[1:]

# def get_headers(dirs):
#     headers = []
#     for d in range(len(dirs)):
#         att_list = []
#         for f in dirs[d][2]:
#             json_path = dirs[d][0]+"/"+f
#             with open(json_path, 'r') as f:
#                 json_data = json.load(f)
#                 att_list = att_list + list(json_data.keys())
#         att_list = list(set(att_list))
#         att_list.sort()
#         headers.append(att_list)
#     return headers

# def write_csv(dirs,headers):
#     for d in range(len(dirs)) :
#         att_list = []
#         csv = []
#         for f in dirs[d][2]:
#             json_path = dirs[d][0]+"/"+f
#             with open(json_path, 'r') as f:
#                 json_data = json.load(f)
#                 row_csv =[]
#                 for h in headers[d]:
#                     if h in json_data.keys():
#                         row_csv.append(json_data[h])
#                         print(json_data[h])
#                     else:
#                         row_csv.append(None)
#             csv.append(row_csv)
        
#         df = pd.DataFrame(csv, columns=headers[d])
#         outdir_result = "./outputs/json_files_to_csv/"+dirs[d][0].split("/")[-1]
#         if not os.path.exists(outdir_result):
#             Path(outdir_result).mkdir(parents=True, exist_ok=True)
#         df.to_csv(outdir_result+"/list.csv", index=False, sep="|")

In [5]:
import os
import pandas as pd
import json
from pathlib import Path
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import json

def profiling(path,source):
    csv_df = pd.read_csv(path,sep="~")
#     json_df = pd.read_json(path,typ='series')
    profile = ProfileReport(csv_df, minimal=True)
    json_data = profile.to_json()
    y = json.loads(json_data)
    profile.to_file("profile_"+source+".json")
#     print(json.dumps(y["variables"], indent=4, sort_keys=True))
    return y["variables"]

def get_dirs(path):
    drs = os.walk(path)
    dirs = []
    for dr in drs:
        dirs.append(dr)
    return dirs[1:]

def get_headers(dirs):
    headers = []
    for d in range(len(dirs)):
        att_list = []
        for f in dirs[d][2]:
            json_path = dirs[d][0]+"/"+f
            with open(json_path, 'r') as f:
                json_data = json.load(f)
                att_list = att_list + list(json_data.keys())
        att_list = list(set(att_list))
        att_list.sort()
        headers.append(att_list)
    return headers

def write_csv(dirs,headers):
    mf_set = []
    mf_header = ["id"]
    for d in range(len(dirs)) :
        att_list = []
        csv = []
        for f in dirs[d][2]:
            json_path = dirs[d][0]+"/"+f
            with open(json_path, 'r') as f:
                json_data = json.load(f)
                row_csv =[]
                for h in headers[d]:
                    if h in json_data.keys():
                        row_csv.append(json_data[h])
#                         print(json_data[h])
                    else:
                        row_csv.append(None)
            csv.append(row_csv)
        
        df = pd.DataFrame(csv, columns=headers[d])
        outdir_result = "./output_monitor2/json_files_to_csv/"+dirs[d][0].split("/")[-1]
        if not os.path.exists(outdir_result):
            Path(outdir_result).mkdir(parents=True, exist_ok=True)
        df.to_csv(outdir_result+"/list.csv", index=False, sep="~")
        source = dirs[d][0].split("/")[-1]
        mf = profiling(outdir_result+"/list.csv",source)
        
        for variable in mf.keys():
            for key in mf[variable].keys():
                if key not in mf_header:
                    mf_header.append(key)
        
        for variable in mf.keys():
            mf_row = [None] * len(mf_header)
            mf_row[0] = source+"//"+variable
            for key in mf[variable].keys():
                i = mf_header.index(key)
                mf_row[i] = mf[variable][key]
            mf_set.append(mf_row)
        
    df = pd.DataFrame(mf_set, columns=mf_header)
    df.to_csv("output_monitor2/"+"attr_mf.csv", index=False, sep="~")

In [6]:
dirs = get_dirs("./2013_monitor_specs/")
headers = get_headers(dirs)
write_csv(dirs,headers)

/home/pablo/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (6,11,12,16,18,32,34,36,42,48,55,70,92,95,99,119,122,125,133,139,147,153,155,169,176,179,185,204,210,213,216,220,226,231,246,256,258,261,272,274,289,292,295,301,302,305,318,321,323,325,334,351,352) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


/home/pablo/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,10,11,12,13,15,16,17,18,19,20,21,23,25,26,30,31,32,33,34,35,36,37,39,41,42,43,45,46,47,48,49,51,52,53,55,57,58,59,60,61,62,65,66,67,68,69,73,74,76,78,81,82,83,84,85,90,91,93,94,96,97,98,100,101,102,103,105,106,107,108,109,110,111,112,115,116,117,118,119,120,122,123,125,126,128,129,130,133,135,139,140,141,142,143,144,145,147,148,149,151,152,156,158,159,160,161,162,163,164,165,166,171,172,174,175,176,180,181,182,183,184,185,187,191,192,193,195,196,197,200,201,202,206,209,213,214,217,220,221,222,223,224,225,227,231,233,234,235,236,237,239,240,241,242,245,246,247,248,249,251,252,255,256,257,259,260,261,262,263,265,266,267,270,271,272,274,276,277,278,279,280,281,282,283,285,286,287,288,289,291,292,295,298,300,301,302,303,307,309,310,313,317,318,319,320,322,325,326,328,329,331,332,336,337,338,339,340,341,342,346,347,348,349,351,352,353,354,355,356,357,359,361,362,364

## Remove duplicates in mapping

In [ ]:
csv_manual = pd.read_csv("./outputs/manual_mappings.csv")
csv_original = pd.read_csv("./outputs/original_mappings.csv")
columns_manual = csv_manual.columns
columns_original = csv_original.columns
print(str("# of original mappings {} \n# of manual mappiings {}").format(len(csv_original[columns_original[0]]),len(csv_manual[columns_manual[0]])))
mappings_original = list([x[0]+"-"+x[1] for x in csv_original.values])
csv_clean = list([x for x in list(csv_manual.values) if x[0]+"-"+x[1] not in mappings_original])    
csv_dups = list([x for x in list(csv_manual.values) if x[0]+"-"+x[1] in mappings_original])    
print(len(csv_clean))
df = pd.DataFrame(csv_clean)
df.to_csv("./outputs/clean_manual_appings.csv", index=False, sep=",",header=["source","target"])

## Get + and - pairs

In [10]:
import itertools
def generate_labelled_pairs(type_set,path):
    csv_manual = pd.read_csv(path)
    targets = {}
    sources = {}
    for row in csv_manual.values:
        t = row[1]
        s = row[0]
        if t not in targets.keys():
            targets[t] =[]
        if s not in sources.keys():
            sources[s] =[]
        targets[t].append(s)
        sources[s].append(t)

    pos_set = []
    pos_csv = []
    for key in targets.keys():
        pos_set = pos_set + list(itertools.combinations(targets[key],2))

    for pair in pos_set:
        pos_csv.append((pair[0],pair[1],1))

    neg_set = []
    processed = []
#     if type_set == "training":
    for source in sources.keys():
        for target in targets.keys():
            if target not in sources[source]:
                for t in targets[target]:
                    if t not in processed:
                        neg_set.append((source,t,0))
        processed.append(source)

    label_set = neg_set + pos_csv         
    df = pd.DataFrame(label_set)
    df.to_csv("./outputs/"+type_set+".csv", index=False, sep=",",header=["id1","id2","relation"])

generate_labelled_pairs("training","./outputs/mapping_manual.csv")
generate_labelled_pairs("test","./outputs/mapiing_original.csv")

In [16]:
import itertools
def group(path):
    csv_manual = pd.read_csv(path)
    targets = {}
    sources = {}
    for row in csv_manual.values:
        t = row[1]
        s = row[0]
        if t not in targets.keys():
            targets[t] =[]
        if s not in sources.keys():
            sources[s] =[]
        targets[t].append(s)
        sources[s].append(t)
        
    return sources,targets

def generate_pairs(s_manual,s_ori,t_manual,t_ori):
    train_pos_set = []
    train_pos_csv = []
    for key in t_manual.keys():
        train_pos_set = train_pos_set + list(itertools.combinations(t_manual[key],2))

    for pair in train_pos_set:
        train_pos_csv.append((pair[0],pair[1],1))

    train_neg_set = []
    processed = []

    for source in s_manual.keys():
        for target in t_manual.keys():
            if target not in s_manual[source]:
                for t in t_manual[target]:
                    if t not in processed:
                        train_neg_set.append((source,t,0))
        processed.append(source)

    train_label_set = train_neg_set + train_pos_csv         
    df = pd.DataFrame(train_label_set)
    df.to_csv("./outputs/monitor_train.csv", index=False, sep=",",header=["id1","id2","relation"])
    
    test_set = []
    for test_source in s_ori.keys():
        for train_source in s_manual.keys():
            if set(s_ori[test_source]).isdisjoint(s_manual[train_source]):
                test_set.append((train_source,test_source,0))
            else:
                test_set.append((train_source,test_source,1))
                    
    df = pd.DataFrame(test_set)
    df.to_csv("./outputs/monitor_test.csv", index=False, sep=",",header=["id1","id2","relation"])
                    
s_manual,t_manual = group("./outputs/mapping_manual.csv")
s_ori,t_ori = group("./outputs/mapiing_original.csv")
generate_pairs(s_manual,s_ori,t_manual,t_ori)

In [27]:
csv_manual = pd.read_csv("./outputs/mapping_manual.csv")
csv_origin = pd.read_csv("./outputs/mapiing_original.csv")

dups = []

for row_m in csv_manual.values:
    duplicates = False
    for row_o in csv_origin.values:
        if row_m[0] == row_o[0]:
            duplicates = True
            dups.append(row_o)
    if duplicates:
        row_m[0] = "__"+row_m[0]
        dups.append(row_m)
        
for d in dups:
    print(d)

['ca.pcpartpicker.com//hdmi' 'hdmi_port_quantity']
['__ca.pcpartpicker.com//hdmi' 'has_hdmi_port']
['www.best-deal-items.com//hd type' 'hd_type']
['__www.best-deal-items.com//hd type' 'has_full_hd']
['www.cleverboxes.com//usb ports' 'usb_port_quantity']
['www.cleverboxes.com//usb ports' 'usb_port_version']
['www.cleverboxes.com//usb ports' 'usb_port2_upstream_quantity']
['__www.cleverboxes.com//usb ports' 'has_usb_port']
['www.cleverboxes.com//dimensions' 'monitor_dimension_with_stand']
['www.cleverboxes.com//dimensions' 'monitor_height_with_stand']
['www.cleverboxes.com//dimensions' 'monitor_width_with_stand']
['www.cleverboxes.com//dimensions' 'monitor_weight_with_stand']
['www.cleverboxes.com//dimensions' 'monitor_depth_with_stand']
['__www.cleverboxes.com//dimensions' 'monitor_depth']
['www.cleverboxes.com//dimensions' 'monitor_dimension_with_stand']
['www.cleverboxes.com//dimensions' 'monitor_height_with_stand']
['www.cleverboxes.com//dimensions' 'monitor_width_with_stand']
['www.

## Exclude repetaed keys

In [26]:
import itertools
def group(path):
    csv_manual = pd.read_csv(path)
    targets = {}
    sources = {}
    for row in csv_manual.values:
        t = row[1]
        s = row[0]
        if t not in targets.keys():
            targets[t] =[]
        if s not in sources.keys():
            sources[s] =[]
        targets[t].append(s)
        sources[s].append(t)
        
    return sources,targets

def generate_pairs(s_manual,s_ori,t_manual,t_ori):
    train_pos_set = []
    train_pos_csv = []
    for key in t_manual.keys():
        train_pos_set = train_pos_set + list(itertools.combinations(t_manual[key],2))

    for pair in train_pos_set:
        train_pos_csv.append((pair[0],pair[1],1))

    train_neg_set = []
    processed = []

    for source in s_manual.keys():
        for target in t_manual.keys():
            if target not in s_manual[source]:
                for t in t_manual[target]:
                    if t not in processed:
                        train_neg_set.append((source,t,0))
        processed.append(source)

    train_label_set = train_neg_set + train_pos_csv         
    df = pd.DataFrame(train_label_set)
    df.to_csv("./outputs/monitor_train.csv", index=False, sep=",",header=["id1","id2","relation"])
    
    test_set = []
    for test_source in s_ori.keys():
        if test_source in s_manual.keys():
            continue
        for train_source in s_manual.keys(): 
            if set(s_ori[test_source]).isdisjoint(s_manual[train_source]):
                test_set.append((train_source,test_source,0))
            else:
                test_set.append((train_source,test_source,1))
                    
    df = pd.DataFrame(test_set)
    df.to_csv("./outputs/monitor_test.csv", index=False, sep=",",header=["id1","id2","relation"])
                    
s_manual,t_manual = group("./outputs/mapping_manual.csv")
s_ori,t_ori = group("./outputs/mapiing_original.csv")
generate_pairs(s_manual,s_ori,t_manual,t_ori)

## Extract metafeatures from files

In [ ]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import json

#print(csv.describe())
#print(csv.info())

#get meta-features from files
csv_pairs = pd.read_csv("./outputs/csv/ca.pcpartpicker.com/list.csv",sep="|")
profile = ProfileReport(csv_pairs, minimal=True)

json_data = profile.to_json()
y = json.loads(json_data)
print(json.dumps(y, indent=4, sort_keys=True))
# print("Number of datasets " + str(y["variables"]["'dataset1_id'"]["distinct_count_without_nan"]))
# print("Number of values per label " + str(y["variables"]["'matching_topic'"]["value_counts"]))
# print("Total of pairs " + str(y["table"]["n"]))
# print("Total of dubplicates " + str(y["table"]["n_duplicates"]))


profile.to_file("your_report.json")

In [ ]:
import numpy as np
import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt
from random import randrange

def read_dataset(path,drop_columns=None,keep_columns=None):
    #get rid of useless columns
    csv_data = pd.read_csv(path)
    
    if keep_columns != None:
        #keep only these columns
        return csv_data.filter(items=keep_columns)
    
    if drop_columns!= None:
        #drop these and keep the rest
        return csv_data.drop(drop_columns, axis=1)
    
    #finally, didn't drop or filter any column
    return csv_data     

def plot_graph(g,ds_nodes=[],attribute_nodes=[],feat_nodes=[],lit_nodes=[]):
    pos=nx.spring_layout(g)    
    nx.draw_networkx_nodes(g,pos,nodelist=ds_nodes,node_color="blue",node_size=900)
    nx.draw_networkx_nodes(g,pos,nodelist=attribute_nodes,node_color="green",node_size=900)
    nx.draw_networkx_nodes(g,pos,nodelist=feat_nodes,node_color="grey",node_size=900)
    nx.draw_networkx_nodes(g,pos,nodelist=lit_nodes,node_color="red",node_size=900)

    nx.draw_networkx_edges(g,pos,width=3)
    nx.draw_networkx_labels(g,pos,font_size=8)
    plt.show() 

## Graph  construction

In [ ]:
def code_ds_id(data):
    return "DS_"+data
def code_attr_id(data,parent):
    return data+"|"+parent
def code_feat_id(data,parent):
    return data+"|"+parent
def code_literal_id(data,parent):
    return "literal_"+data+"|"+parent

In [ ]:
def graph_dataset(datasets,g=None,wem="fasttext",instances=0):
    if g == None:
        g = nx.Graph()
    
    #create nodes and edges at datasetLevel
    features = datasets.columns[1:]
    
    if instances==0:
        number_instances = len(datasets)
    else:
        number_instances = instances
    
    for r in range(number_instances): 
        #node id is the openML id which is in the first column
        dataset_id = code_ds_id(str(datasets.iloc[r][0]))
        g.add_node(dataset_id,vector=word_embedding("dataset",wem),tipo="dataset")
        row = datasets.iloc[r][1:]
        
        if instances == 0:
            number_features = len(features)
        else:
            number_features = min(instances,len(features))
            
        for i in range (number_features):
            feature_dataset_id = code_feat_id(features[i],dataset_id)
            literal_dataset_id = code_literal_id(str(i),dataset_id)
            g.add_node(feature_dataset_id,vector=word_embedding("feature dataset|" +features[i] ,wem),tipo="feature dataset")
            g.add_node(literal_dataset_id,vector=word_embedding(row[i],wem),tipo="literal dataset")
            g.add_edge(dataset_id,feature_dataset_id)
            g.add_edge(feature_dataset_id,literal_dataset_id)
            
    return g


def graph_attribute(datasets,g=None,wem="fasttext",instances=0):
    if g == None:
        g = nx.Graph()
        
    #create nodes and edges at datasetLevel
    features = datasets.columns[2:]
    
    if instances==0:
        number_instances = len(datasets)
    else:
        number_instances = min (instances,len(datasets))
    
    for r in range(number_instances): 
        #node id is the openML id which is in the first column
        #attr name is the 2nd column
        dataset_id = code_ds_id(str(datasets.iloc[r][0]))
        attribute_id = code_attr_id(datasets.iloc[r][1],dataset_id)
        row = datasets.iloc[r][2:]
        
        g.add_node(attribute_id,vector=word_embedding("attribute",wem),tipo="attribute")
        
        #relation of dataset and an attribute
        g.add_edge(dataset_id,attribute_id)
        
        if instances == 0:
            number_features = len(features)
        else:
            number_features = min (instances,len(features))
            
        for i in range (number_features):
            feature_attribute_id = code_feat_id(features[i],attribute_id)
            literal_dataset_id = code_literal_id(str(i),attribute_id)
            g.add_node(feature_attribute_id,vector=word_embedding("feature attribute|"+features[i],wem),tipo="feature attribute")
            g.add_node(literal_dataset_id,vector=word_embedding(row[i],wem),tipo="literal attribute")
            g.add_edge(attribute_id,feature_attribute_id)
            g.add_edge(feature_attribute_id,literal_dataset_id)
            
    return g

## Auxiliars

### Check if input is number

In [ ]:
def is_number(s):
    #Returns True is string is a number.
    try:
        float(s)
        return True
    except ValueError:
        return False

### From numbers to bin tensor vector

In [ ]:
from decimal import Decimal
import bitstring
def num2vec(num):
    
    rep_sc = str('{:.11E}'.format(num))
    
    dec_part = int(rep_sc.split("E")[0].replace(".",""))
    c = 1
    if dec_part <0:
        c = -1
    dec_part = abs(dec_part)
    
    exp_part = int(rep_sc.split("E")[1])
    if exp_part <0:
        exp_pos = 0
        exp_neg = exp_part
    else:
        exp_pos = exp_part
        exp_neg = 0

    exp_pos = abs(exp_pos)    
    exp_neg = abs(exp_neg)
    
    rep_str = str("{:03}{:03}{:012}".format(exp_pos,exp_neg,dec_part))
    
#     print(dec_part)
    rep_int = int(rep_str) * c
    rep_bin = bitstring.Bits(int=rep_int, length=64).bin

    bin_tensor = torch.tensor(np.array([float(x) for x in rep_bin]))
    return bin_tensor

## Fasttext

In [ ]:
import numpy as np
import torch
import fasttext
#fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('./resources/fasttext.bin')
print(ft.get_dimension())


In [ ]:
def fasttex_(value):
    if is_number(value):
        value_f = float(value)
        bin_tensor = num2vec(value_f)
        return torch.cat((torch.zeros(300),bin_tensor.float()))
    else:
        values = value.split("|")
        out_tensor = torch.zeros(300)
        for v in values:
            out_tensor = out_tensor + torch.tensor(ft.get_sentence_vector(value))
        out_tensor = out_tensor / len(values)
        return torch.cat((out_tensor.float(),torch.zeros(64)))

## Bert


In [ ]:
#pip install torch==1.4.0+cpu torchvision==0.5.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
#pip install transformers
from transformers import BertModel, BertTokenizer
import torch

#load model in memory
tokenizer = BertTokenizer.from_pretrained('google/bert_uncased_L-12_H-768_A-12')
model = BertModel.from_pretrained('google/bert_uncased_L-12_H-768_A-12')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained('bert-base-uncased')
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

def bert(value):
    if is_number(value):
        value = float(value)
        bin_tensor = num2vec(value)
        return torch.cat((torch.zeros(768),bin_tensor.float()))
    else:
        values = value.split("|")
        out_tensor = torch.zeros(768)
        for v in values:
            #add special tokens at the begining and end, and takes until 512 tokens max 
            tokenized = tokenizer.encode(v, add_special_tokens=True,max_length=512)
            input_ids = torch.tensor(tokenized).unsqueeze(0)  # Batch size 1
            outputs = model(input_ids)

            last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
            #result shape: (batch size, sequence length, model hidden dimension)
#             print(last_hidden_states.shape)

            #make the mean of the vectors to have 1 vector for the whole sentence and store result
            out_tensor = out_tensor + torch.mean(last_hidden_states[0],dim=0).detach()
            
        out_tensor = out_tensor / len(values)
        return torch.cat((out_tensor.float(),torch.zeros(64)))

## Choose word embedding

In [ ]:
def word_embedding(data, model):
    if model=="fasttext":
        return fasttex_(data)
    if model=="bert":
        return bert(data)

# Execute

In [ ]:
#build graph
df_dataset = read_dataset("./resources/openml_203ds_datasets_index.csv",drop_columns=["Num", "dataset_topic"]);
g = g = nx.Graph()
g = graph_dataset(df_dataset,g,"bert")
df_attributes = read_dataset("./resources/openml_203ds_attributes_nominal_index.csv",drop_columns=["dataset_name", "type_converted"]);
g = graph_attribute(df_attributes,g,"bert")
df_attributes_numeric = read_dataset("./resources/openml_203ds_attributes_numeric_index.csv",drop_columns=["dataset_name", "type_converted"]);
g = graph_attribute(df_attributes_numeric,g,"bert")

#write graph to file
nx.write_gpickle(g, "../word_embeddings/encoded_bert_v2.gpickle")

## Read previously created graph

In [ ]:
#read
g = nx.read_gpickle("../word_embeddings/encoded_bert_v2.gpickle")

## Deprecated

In [ ]:
datasets = [x for x,y in g.nodes(data=True) if y['tipo']=="dataset"]
datasets_features = [x for x,y in g.nodes(data=True) if y['tipo']=="feature dataset"]
datasets_literals = [x for x,y in g.nodes(data=True) if y['tipo']=="literal dataset"]
attributes = [x for x,y in g.nodes(data=True) if y['tipo']=="attribute"]
attributes_features = [x for x,y in g.nodes(data=True) if y['tipo']=="feature attribute"]
attributes_literals = [x for x,y in g.nodes(data=True) if y['tipo']=="literal attribute"]

In [ ]:
print(len(datasets))
print(len(datasets_features))
print(len(datasets_literals))
print(len(attributes))
print(len(attributes_features))
print(len(attributes_literals))

In [ ]:
#plot
datasets = [x for x,y in g.nodes(data=True) if y['tipo']=="dataset"]
datasets_features = [x for x,y in g.nodes(data=True) if y['tipo']=="feature dataset"]
datasets_literals = [x for x,y in g.nodes(data=True) if y['tipo']=="literal dataset"]
attributes = [x for x,y in g.nodes(data=True) if y['tipo']=="attribute"]
attributes_features = [x for x,y in g.nodes(data=True) if y['tipo']=="feature attribute"]
attributes_literals = [x for x,y in g.nodes(data=True) if y['tipo']=="literal attribute"]
# plot_graph(g,datasets,attributes,datasets_features + attributes_features,datasets_literals + attributes_literals);

## Get nodes and print data from nodes

In [ ]:
sample = [y for x,y in g.nodes(data=True) if y['tipo']=="dataset"]
print(sample[0]["vector"])